In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import json
import plotly.express as px
import matplotlib as plt
import random
import pickle
import gzip
import networkx as nx
from data_handler import DataHandler

In [ ]:
handler = DataHandler()
chicago = handler.load_chicago()
#centers = handler.load_centers()
graph = handler.load_graph_new()

In [4]:
#chicago = gpd.read_file('data/chicago.json')
#center_lat, center_lon = projection(chicago)  # see gerrychain library for that function. 
#fig =  plot.fig_blocks(chicago)

# Dual Graph of Census Blocks

In [ ]:
""" We use Gerrychain library to construct a dual graph of Census blocks. """

# Write a function to create a graph removing default/secified columns from a dataset

#graph = Graph.from_geodataframe(chicago)
#nx.draw(graph,pos={node:(graph.nodes()[node]['centroid'].x,graph.nodes()[node]['centroid'].y) for node in graph.nodes()}, node_size=20)
#graph.to_json("GraphToJson.json")

# graph = Graph.from_json("data/GraphToJson.json")

# Opening JSON file
#f = open('GraphToJson.json')
# returns JSON object as a dictionary
#data = json.load(f)
#f.close()
#data.keys()

"""adjacency = random.choice(list(data['adjacency']))
adjacency"""

"""node = random.choice(list(graph.nodes))
graph.nodes[node]['GEOID20'] """

"""first_node = list(graph.nodes)[0]
node_attributes = graph.nodes[first_node]
print(f"Attributes for node {first_node}: {node_attributes}")"""

""" lookup()  #Lookup a node/field attribute. Returns the value of the attribute field at node.""" 

- The areas of the polygons are included as node attributes (with key `area`).
        
- The shared perimeter of neighboring polygons are included as edge attributes (with key `shared_perim`).

- Nodes corresponding to polygons on the boundary of the union of all the geometries (e.g., the state, if your dataframe describes VTDs) have a    `boundary_node` attribute (set to `True`) and a `boundary_perim` attribute
with the length of this "exterior" boundary.

- Gerrychain library has centroid function.

# Get Populations from DHC Data

In [ ]:
#dhc = pd.read_csv('data/dhc_2020.csv')
#pop_dict = load.get_column(data = dhc, column = 'TOT_POP')  # returns {key: geoid, value: population}
#load.add_to_graph(graph, pop_dict)  # adds 'population' attributes to nodes
#load.add_to_data(chicago, pop_dict)

See in API that

classgerrychain.partition.GeographicPartition

crosses_parts(edge: Tuple)→ bool

flip(flips: Dict)→ Partition

plot(geometries=None, **kwargs)

# Generate Random PHCs

In [ ]:
#phc_df, selected_blocks = load.random_phc(chicago)
#candidates = load.add_phc(chicago, graph, selected_blocks)
#fig = plot.add_phc(fig, phc_df)
#fig.show() 

# Calculate Travel Times

NOTE: Correct the projection to get the precise walking times.

In [2]:
""""travel_times calculated and saved. Travel time betwween two blocks is fixed as 10 minutes."
handler = DataHandler()
graph = handler.load_graph()
candidates = handler.load_candidates()

import pickle
import gzip 

file_path = '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/newtravel.pkl.gz'
with gzip.open(file_path, 'wb') as file:
                pickle.dump(t, file)

for u, v in graph.edges():
    graph[u][v]['time'] = 10
t = travel.travel_time(graph, candidates)"""

In [5]:
"Travel Time Mode: Walking Time with the Shortest Distance"

import math

# the shortest distance in km over the earth’s surface using the ‘Haversine’ formula.
def get_distance_from_lat_lon_in_km(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the earth in km
    dLat = deg2rad(lat2 - lat1)
    dLon = deg2rad(lon2 - lon1)
    a = (
        math.sin(dLat / 2) * math.sin(dLat / 2) +
        math.cos(deg2rad(lat1)) * math.cos(deg2rad(lat2)) *
        math.sin(dLon / 2) * math.sin(dLon / 2)
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c # Distance in km
    return d

def deg2rad(deg):
    return deg * (math.pi / 180)

speed = 4.5  # walking speed in km/h for pedestrians (considering traffic lights. See the the urban design section on wiki page of Preferred_walking_speed: typical 5.0 km/h, recommended 4.8 km/h)
# Calculate distances between block centroids and assign them as edge weights
for edge in graph.edges:
    u, v = edge
    lat1, lon1 = graph.nodes[u]['C_Y'], graph.nodes[u]['C_X']
    lat2, lon2 = graph.nodes[v]['C_Y'], graph.nodes[v]['C_X']
    
    weight_hours = get_distance_from_lat_lon_in_km(lat1, lon1, lat2, lon2) / speed # walking time in hours
    weight = weight_hours * 60 * 60  # convert to seconds
    
    graph.edges[edge]['walk'] = math.floor(weight) # no need for miliseconds

In [3]:
edge = random.choice(list(graph.edges))
graph.edges[edge]

{'shared_perim': 0.00045513733859732183,
 'time': 10,
 'random_weight': 0.07680618364019531,
 'walk': 107}

In [8]:
path = '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/graph_new.pkl'
with open(path, 'wb') as file:
                pickle.dump(graph, file)

In [4]:
candidates = handler.load_candidates()

import pickle
import gzip 

attr = 'walk'

t = travel.travel_time(graph, candidates, attr=attr)

file_path = '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/travel_walking.pkl.gz'
with gzip.open(file_path, 'wb') as file:
                pickle.dump(t, file)

# Save Objects

In [ ]:
" 1) Graph is saved as graph.pkl in prepared_data folder."
# load.cache_data(graph, file_path='prepared_data/graph.pkl', method = 'pickle')  
" Surprisingly, it uses only 10MB memory, while GraphToJson.json uses 27MB."

' Surprisingly, it uses only 10MB memory, while GraphToJson.json uses 27MB.'

In [ ]:
""" 2) Travel times is saved as travel_times.pkl.gz
    ujson is loaded in 35 sec, and uses 140MB 
    pkl.gz is loaded in 16-17 sec, and uses 130MB memory. """
# load.cache_data(travel_times, file_path='prepared_data/travel_times.pkl.gz', method = 'pickle_gzip')

In [ ]:
""" 3) Chicago data is saved as chicago.pkl.
    pkl is loaded in 1 sec, and uses 19MB memory. 
    Original chicago.json file uses 38MB.
    I didn't really get it. pkl is better than json in both memory and loading."""
    
# load.cache_data(chicago, file_path='prepared_data/chicago.pkl', method='pickle')

In [ ]:
""" 4) Possible PHC locations (candidates) are saved as candidates.pkl"""
# load.cache_data(candidates, file_path='prepared_data/candidates.pkl', method='pickle')

# Upload Real Libraries and Schools Data (Real Facility Candidates)

The coordinates are currently in tuple form inside a string. We need to convert these into a Point geometry object from the shapely.geometry module.

In [3]:
handler = DataHandler()
#vars(handler)

In [4]:
chicago = handler.load_chicago()
libraries = handler.load_libraries()
schools = handler.load_geo_schools ()
candidates = handler.load_candidates()

In [6]:
chicago.columns

Index(['id', 'STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'BLOCKCE20', 'GEOID20',
       'NAME20', 'MTFCC20', 'UR20', 'UACE20', 'UATYPE20', 'FUNCSTAT20',
       'ALAND20', 'AWATER20', 'INTPTLAT20', 'INTPTLON20', 'geometry',
       'centroid', 'constant', 'pop', 'phc'],
      dtype='object')

In [7]:
graph = Graph.from_geodataframe(chicago)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gerrychain/graph/graph.py:179: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()


In [ ]:
districts.to_crs(districts.estimate_utm_crs(), inplace=True)
units.to_crs(units.estimate_utm_crs()), inplace=True)

In [8]:
import random
node = random.choice(list(graph.nodes))
graph.nodes[node]

{'boundary_node': False,
 'area': 2.2087594999952947e-06,
 'id': '318322',
 'STATEFP20': '17',
 'COUNTYFP20': '031',
 'TRACTCE20': '062400',
 'BLOCKCE20': '1011',
 'GEOID20': '170310624001011',
 'NAME20': 'Block 1011',
 'MTFCC20': 'G5040',
 'UR20': 'R',
 'UACE20': '',
 'UATYPE20': '',
 'FUNCSTAT20': 'S',
 'ALAND20': 20345,
 'AWATER20': 0,
 'INTPTLAT20': '+41.9401691',
 'INTPTLON20': '-087.6674627',
 'geometry': <POLYGON ((-87.669 41.941, -87.669 41.94, -87.669 41.94, -87.669 41.94, -87....>,
 'centroid': <POINT (-87.667 41.94)>,
 'constant': 2,
 'pop': 115.0,
 'phc': 0}

In [10]:
edge = random.choice(list(graph.edges))
graph.edges[edge]

{'shared_perim': 0.0018255796339829787}

In [ ]:
"""libraries = gpd.read_file('/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/data/libraries.csv') """

In [5]:
from shapely import Point

# Convert string to Point geometry
def convert_to_point(location_str):
    lat, lon = eval(location_str)
    return Point(lon, lat)

# Apply the conversion function to create the geometry column
libraries['geometry'] = libraries['LOCATION'].apply(convert_to_point)

# Convert DataFrame to GeoDataFrame
geo_libraries = gpd.GeoDataFrame(libraries, geometry='geometry')

# Now you can work with `geo_libraries` as a GeoDataFrame
print(geo_libraries)

                             NAME   \
0                  Vodak-East Side   
1                      Albany Park   
2                           Avalon   
3                         Brainerd   
4   Popular Library at Water Works   
..                             ...   
76        Woodson Regional Library   
77                         Mayfair   
78                     Chicago Bee   
79                          Uptown   
80                    Sherman Park   

                                   HOURS OF OPERATION  \
0   Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...   
1   Mon. & Wed., 10-6; Tues. & Thurs., Noon-8; Fri...   
2   Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...   
3   Mon. & Wed., 10-6; Tues. & Thurs., Noon-8; Fri...   
4    Mon. - Thurs., 10-6; Fri. & Sat., 9-5; Sun., 1-5   
..                                                ...   
76    Mon. - Thurs., 9-8; Fri. & Sat., 9-5; Sun., 1-5   
77  Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...   
78  Mon. & Wed., Noon-8; Tues.

In [6]:
schools.head()

,id,Short_Name,Address,Grades,Lat,Long,COMMAREA,geometry
0,400009,GLOBAL CITIZENSHIP,4647 W 47TH ST,"K, 1, 2, 3, 4, 5, 6, 7, 8",41.807579,-87.740097,GARFIELD RIDGE,POINT (-87.74010 41.80758)
1,400010,ACE TECH HS,5410 S STATE ST,"9, 10, 11, 12",41.796122,-87.625849,WASHINGTON PARK,POINT (-87.62585 41.79612)
2,400011,LOCKE A,3141 W JACKSON BLVD,"K, 1, 2, 3, 4, 5, 6, 7, 8",41.877248,-87.705235,EAST GARFIELD PARK,POINT (-87.70523 41.87725)
3,400013,ASPIRA - EARLY COLLEGE HS,3986 W BARRY AVE,"9, 10, 11, 12",41.937298,-87.727096,AVONDALE,POINT (-87.72710 41.93730)
4,400017,ASPIRA - HAUGAN,3729 W LELAND AVE,"6, 7, 8",41.966406,-87.721825,ALBANY PARK,POINT (-87.72182 41.96641)


In [ ]:
#candidates = load.add_phc(chicago, graph, selected_blocks)
#fig = plot.add_phc(fig, phc_df)
#fig.show()

In [7]:
graph = handler.load_graph()

node = random.choice(list(graph.nodes))
graph.nodes[node]

{'boundary_node': False,
 'area': 5.215014999926553e-07,
 'STATEFP20': '17',
 'COUNTYFP20': '031',
 'TRACTCE20': '160800',
 'BLOCKCE20': '4011',
 'GEOID20': '170311608004011',
 'ALAND20': 4803,
 'C_X': -87.71486363395856,
 'C_Y': 41.95047205837281,
 'pop': 27,
 'phc': 0}

In [9]:
chicago['id']

0         22984
1         22985
2         22986
3         22987
4         22989
          ...  
39516    362595
39517    362596
39518    362732
39519    363152
39520    363197
Name: id, Length: 39521, dtype: object

In [10]:
from shapely.geometry import Point, Polygon

In [11]:
geometries = {}
for index, row in chicago.iterrows():
    geometries[index] = row['geometry']

In [12]:
for node in graph.nodes:
    # Check if the node exists in the geometries dictionary
    if node in geometries:
        if any(geometries[node].contains(point) for point in schools['geometry']):
            graph.nodes[node]['real_phc'] = True
        else:
            graph.nodes[node]['real_phc'] = False
    else:
        # Handle the case where no geometry is defined for the node
        graph.nodes[node]['real_phc'] = False
        print(f"No geometry found for node {node}")


In [13]:
# Node shapely geometry
for node in graph.nodes:
        
    if any(geometries[node].contains(point) for point in geo_libraries['geometry']):
        graph.nodes[node]['real_phc'] = True

In [14]:
count = 0 
for node in graph.nodes:
    if graph.nodes[node]['real_phc']==True:
        count+=1
        
count

693

In [13]:
node = random.choice(list(graph.nodes))
graph.nodes[node]

{'boundary_node': False,
 'area': 7.904144999979941e-07,
 'STATEFP20': '17',
 'COUNTYFP20': '031',
 'TRACTCE20': '840200',
 'BLOCKCE20': '1041',
 'GEOID20': '170318402001041',
 'ALAND20': 7291,
 'C_X': -87.6424472502809,
 'C_Y': 41.845612936281114,
 'pop': 67,
 'phc': 0,
 'real_phc': False}

In [2]:
from data_utils import DataHandler
handler = DataHandler()

lib = handler.load_geo_libraries()
sch = handler.load_geo_schools()

In [61]:
# Concatenate DataFrames along rows (axis=0)
merged = pd.concat([sch, lib], axis=0, ignore_index=True)




In [26]:
file_path = '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/geo_libraries.pkl'
with open(file_path, 'wb') as file:
                pickle.dump(geo_libraries, file)

In [68]:
import pickle
file_path = '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/geo_candidates.pkl'
with open(file_path, 'wb') as file:
                pickle.dump(merged, file)

# Artificial Candidates

In [ ]:
# Lately, we relaize that there is no enough candidates aroung Chicago O'Hare Airport.
# We select some census blocks manually from the map, save their centroids as artifical candidates
# and merge them into geo_canidates.

In [3]:
import pickle

handler = DataHandler()
graph = handler.load_graph()
newtravel = handler.load_newtravel()
chicago = handler.load_chicago()
geo_candidates = handler.load_geo_candidates()
candidates = handler.load_candidates()

In [ ]:
"Artificial candidates"
artificial_candidates = {32681, 516, 2973, 24279, 29, 32306, 8289}

# Create a new GeoDataFrame for artificial_candidates
artificial_candidates_data = {
    'block': list(artificial_candidates),
    'geometry': [chicago.loc[block, 'centroid'] for block in artificial_candidates]
}

geo_artificial_candidates = gpd.GeoDataFrame(artificial_candidates_data, geometry='geometry')

geo_candidates['artificial'] = False
geo_artificial_candidates['artificial'] = True 

# Set the CRS for geo_artificial_candidates
geo_artificial_candidates.set_crs(geo_candidates.crs, inplace=True)

# Concatenate the new GeoDataFrame with geo_candidates
geo_candidates = pd.concat([geo_candidates, geo_artificial_candidates], ignore_index=True)

# Reset index of geo_candidates if needed
geo_candidates.reset_index(drop=True, inplace=True)

import pickle
file_path = '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/geo_artificial_candidates.pkl'
with open(file_path, 'wb') as file:
                pickle.dump(geo_artificial_candidates, file)

file_path = '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/geo_candidates.pkl'
with open(file_path, 'wb') as file:
                pickle.dump(geo_candidates, file)

In [ ]:
# edit other datasets              
for node in artificial_candidates:
    graph.nodes[node]['real_phc'] = True

file_path = '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/ggraph.pkl'
with open(file_path, 'wb') as file:
                pickle.dump(graph, file)


from travel_time import travel_time_to_source

for source in artificial_candidates:
    to_source = travel_time_to_source(graph, source)
    for i in graph.nodes:
        newtravel[(i, source)] = to_source[i]
        
file_path = '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/newtravel.pkl.gz'
with open(file_path, 'wb') as file:
                pickle.dump(newtravel, file)
                
candidates = candidates.union(artificial_candidates)
file_path = '/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/candidates.pkl'
with open(file_path, 'wb') as file:
                pickle.dump(candidates, file)

# Scale Areas 

In [ ]:
# We scale utm areas in such a way that density of a spanning tree is 1.

def scale_areas(n_teams, pop_col=None):
    # change the name of the population attribute
    pop_target = pop / n_teams
    pop = sum(graph.nodes[node][pop_col] for node in graph.nodes)
    sum_utm = sum(graph.nodes[node]['area'] for node in graph.nodes)
    factor = pop / sum_utm

    for node in graph.nodes:
        graph.nodes[node]['area_scaled'] = graph.nodes[node]['area']*factor 
        
    return